In [23]:
import gzip
import json
import pandas as pd
from my_functions_improved import *

In [24]:
with gzip.open('noun_dict.json.gz', 'rt', encoding='utf-8') as gzip_file:
    noun_dict = json.load(gzip_file)

In [25]:
with gzip.open('verb_dict.json.gz', 'rt', encoding='utf-8') as gzip_file:
    verb_dict = json.load(gzip_file)

In [26]:
noun_df = pd.DataFrame(noun_dict)

In [27]:
unique_values = noun_df['number'].unique()
unique_values

array(['dual', 'sg', 'pl'], dtype=object)

In [28]:
noun_df["form"]

0              ἅβρα
1              ἅβρα
2             ἅβραι
3             ἅβραι
4            ἅβραις
            ...    
216247    ῐ̔́στορες
216248      ῐ̔́στωρ
216249     ῐ̔́στορε
216250    ῐ̔́στορες
216251      ῐ̔́στορ
Name: form, Length: 216252, dtype: object

In [85]:
pd.set_option('display.max_rows', None)  # This will allow all rows to be displayed.
pd.set_option('display.max_columns', None)

In [88]:
l = verb_df.loc[verb_df["lemma"]=="ὀχέω"]
l

,lemma,form,tense,mode,act/mid/p,gender,case,person,number,dialects
563036,ὀχέω,ὄχει,pres,imperat,act,,,2nd,sg,
563037,ὀχέω,ὄχει,imperf,ind,act,,,3rd,sg,
563038,ὀχέω,ὄχεσκον,imperf,ind,act,,,3rd,pl,[ionic]
563039,ὀχέω,ὄχεσκον,imperf,ind,act,,,1st,sg,[ionic]
563040,ὀχέω,ὄχη,pres,imperat,act,,,2nd,sg,"[(doric, aeolic]"
...,...,...,...,...,...,...,...,...,...,...
563169,ὀχέω,ὠχήσατο,aor,ind,mid,,,3rd,sg,
563170,ὀχέω,ὠχημένος,perf,part,mp,masc,nom,,sg,
563171,ὀχέω,ὠχούμην,imperf,ind,mp,,,1st,sg,[doric]
563172,ὀχέω,ὠχοῦ,imperf,ind,mp,,,2nd,sg,


In [87]:
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

In [30]:
verb_df = pd.DataFrame(verb_dict)

In [31]:
verb_df

,lemma,form,tense,mode,act/mid/p,gender,case,person,number,dialects
0,ἅλλομαι,ἅλεται,aor,subj,mid,,,3rd,sg,
1,ἅλλομαι,ἅληται,aor,subj,mid,,,3rd,sg,
2,ἅλλομαι,ἅλῃ,aor,subj,mid,,,2nd,sg,
3,ἅλλομαι,ἅλλεσθαι,pres,inf,mid,,,,,
4,ἅλλομαι,ἅλλεσθε,imperf,ind,mid,,,2nd,pl,"[(doric, aeolic]"
...,...,...,...,...,...,...,...,...,...,...
859327,ζῳοτροφέω,ζῳοτροφεῖν,pres,inf,act,,,,,[doric]
859328,ζῳοτροφέω,ζῳοτροφούντων,pres,part,act,masc/neut,gen,,pl,[doric]
859329,ζῳοτροφέω,ζῳοτροφούντων,pres,imperat,act,,,3rd,pl,[doric]
859330,ζῳοτροφέω,ζῳοτροφοῦσιν,pres,part,act,masc/neut,dat,,pl,[doric]


In [32]:
strings = ["ἂν δὲ καὶ αὐτοὶ βάντες ἐπὶ κληῖσι καθῖζον", "ἑξῆς δ᾽ ἑζόμενοι πολιὴν ἅλα τύπτον ἐρετμοῖς", "ἔνθεν δὲ προτέρω πλέομεν ἀκαχήμενοι ἦτορ", "ἄσμενοι ἐκ θανάτοιο, φίλους ὀλέσαντες ἑταίρους", "νῆα μὲν οἵ γε μέλαιναν ἁλὸς βένθοσδε ἔρυσσαν", "τοὶ δὲ πρυμνήσι᾽ ἔλυσαν"]

In [33]:
def get_noun_analysis(noun, df):
    noun_analysis = []
    word_df = df.loc[df["form"]==noun]
    for id, row in word_df.iterrows():
        noun_analysis.append([row["gender"], row["case"], row["number"], row ["dialects"]])
    return noun_analysis

In [34]:
def get_verb_analysis(verb, df):
    verb_analysis = []
    word_df = df.loc[df["form"]==verb]
    for id, row in word_df.iterrows():
        verb_analysis.append([row["tense"], row["mode"], row["act/mid/p"], row ["gender"], row["case"], row["person"], row["number"], row["dialects"]])
    return verb_analysis

In [35]:
import pickle

In [36]:
with open("sents_sim_dict.pickle", "rb") as file:
    sents_sim = pickle.load(file)

In [57]:
def get_lemma_syn(word, sim_dicts):
    lemma_w = lemmatize_grecy(word)[0]
    
    for sim_dict in sim_dicts:
        for lemma in sim_dict:
            if lemma == lemma_w:
                if sim_dict[lemma] != []:
                    sorted_values = sorted(sim_dict[lemma], key=lambda x: x[1], reverse=True)
                    for i, tup in enumerate(sorted_values):
                        syn, value = sorted_values[i]
                        if syn not in verb_df["lemma"].values and syn not in noun_df["lemma"].values:
                            continue
                        else:
                            return syn

In [45]:
noun_df

,lemma,form,gender,case,number,dialects
0,ἅβρα,ἅβρα,fem,nom/voc/acc,dual,
1,ἅβρα,ἅβρα,fem,nom/voc,sg,"[(attic, doric, aeolic]"
2,ἅβρα,ἅβραι,fem,nom/voc,pl,
3,ἅβρα,ἅβραι,fem,dat,sg,"[(attic, doric, aeolic]"
4,ἅβρα,ἅβραις,fem,dat,pl,
...,...,...,...,...,...,...
216247,ἵστωρ,ῐ̔́στορες,,nom,pl,
216248,ἵστωρ,ῐ̔́στωρ,,nom,sg,
216249,ἵστωρ,ῐ̔́στορε,,voc,dual,
216250,ἵστωρ,ῐ̔́στορες,,voc,pl,


In [79]:
def get_infl_s_noun(noun, analysis):
    forms = []
    if analysis[3] != " ":
        for dialect in analysis[3]:
            syn_df = noun_df.loc[(noun_df["lemma"] == noun) & ((noun_df["gender"] == analysis[0]) | (noun_df["gender"] == " ")) & (noun_df["case"] == analysis[1]) & (noun_df["number"] == analysis[2]) & (noun_df["dialects"] == dialect)]
            for id, row in syn_df.iterrows():
                forms.append(row["form"])
    else:
        syn_df = noun_df.loc[(noun_df["lemma"] == noun) & ((noun_df["gender"] == analysis[0]) | (noun_df["gender"] == " ")) & (noun_df["case"] == analysis[1]) & (noun_df["number"] == analysis[2])]
        for id, row in syn_df.iterrows():
            forms.append(row["form"])
    return forms

In [80]:
def get_infl_s_verb(verb, analysis):
    forms = []
    if analysis[7] != " ":
        for dialect in analysis[7]:
            syn_df = verb_df.loc[(verb_df["lemma"] == verb) & (verb_df["tense"] == analysis[0]) & (verb_df["mode"] == analysis[1]) & (verb_df["act/mid/p"] == analysis[2]) & (verb_df["gender"] == analysis[3]) & (verb_df["case"] == analysis[4]) & (verb_df["person"] == analysis[5]) & (verb_df["number"] == analysis[6]) & (verb_df["dialects"] == dialect)]
            for id, row in syn_df.iterrows():
                forms.append(row["form"])
    else:
        syn_df = verb_df.loc[(verb_df["lemma"] == verb) & (verb_df["tense"] == analysis[0]) & (verb_df["mode"] == analysis[1]) & (verb_df["act/mid/p"] == analysis[2]) & (verb_df["gender"] == analysis[3]) & (verb_df["case"] == analysis[4]) & (verb_df["person"] == analysis[5]) & (verb_df["number"] == analysis[6])]
        for id, row in syn_df.iterrows():
            forms.append(row["form"])
    return forms

In [76]:
def get_infl_syn(lemma, analysis):
    pos = pos_tag_grecy(lemma)[0]
    if pos == "a" or pos == "n":
        forms = get_infl_s_noun(lemma, analysis)
    if pos == "v":
        forms = get_infl_s_verb(lemma, analysis)
    return forms


In [54]:
s = get_lemma_syn("βάντες", sents_sim)
s

βαίνω


'διαβαίνω'

In [42]:
for sim_dict in sents_sim:
    report_similarities(sim_dict)

The best synonyms for βαίνω are:
	διαβαίνω with a score of [[0.88808715]]
	ἕρπω with a score of [[0.87936157]]
	στείχω with a score of [[0.8118147]]
	πατέω with a score of [[0.]]
The best synonyms for καθίζω are:
	ἵζω with a score of [[0.94065994]]
	καταδύω with a score of [[0.94029063]]
	κατατίθημι with a score of [[0.9275684]]
	ἰάλλω with a score of [[0.91808397]]
	εὐνάω with a score of [[0.89377445]]
	τίθημι with a score of [[0.89339024]]
	ἐφίστημι with a score of [[0.86465937]]
	ὑφίστημι with a score of [[0.81488335]]
	πέμπω with a score of [[0.7911981]]
	ἐπιτίθημι with a score of [[0.7376522]]
	λέγω with a score of [[0.6852222]]
	συγκαλέω with a score of [[0.4999244]]
	καθίστημι with a score of [[0.3065406]]
	συνάγω with a score of [[0.]]
	προτίθημι with a score of [[0.]]
	ἰάπτω with a score of [[0.]]
	ὑπομιμνήσκω with a score of [[0.]]
	ἐπικλίνω with a score of [[0.]]
	ὑποστόρνυμι with a score of [[0.]]
	στηρίζω with a score of [[0.]]
The best synonyms for ἕζομαι are:
	ἵζω with a

In [75]:
print(pos_tag_cltk("ἐνέπω")[0])

INTJ


In [84]:
# for s in strings:
s = 'ἔνθεν δὲ προτέρω πλέομεν ἀκαχήμενοι ἦτορ'
paraphrasis = []
sent_split = s.split(" ")
for id, word in enumerate(sent_split):
    new_sent_split = sent_split.copy()
    word_analysis = []
    pos = pos_tag_grecy(word)[0]
    if pos == "a" or pos == "n":
        word_analysis = get_noun_analysis(word, noun_df)
    if pos == "v":
        word_analysis = get_verb_analysis(word, verb_df)
    if word_analysis != []:
        for analysis in word_analysis:
            lemma_synonym = get_lemma_syn(word, sents_sim)
            if lemma_synonym != None:
                print(lemma_synonym, analysis)
                inflected_syns = get_infl_syn(lemma_synonym, analysis)
                for syn in inflected_syns:
                    new_sent_split[id] = syn
                    new_sent = " ".join(new_sent_split)
                    if new_sent not in paraphrasis:
                        paraphrasis.append(new_sent)
print(paraphrasis)

            

ὀχέω ['pres', 'ind', 'act', ' ', ' ', '1st', 'pl', ' ']
ὀχέω ['imperf', 'ind', 'act', ' ', ' ', '1st', 'pl', ['(homeric', 'ionic']]
[]


In [ ]:
get_infl_syn("ὀχέω")

In [82]:
strings[2]

'ἔνθεν δὲ προτέρω πλέομεν ἀκαχήμενοι ἦτορ'

In [63]:
verb_df.loc[(verb_df["lemma"] == "ἐνέπω") & (verb_df["tense"] == 'aor') & (verb_df["mode"] == 'ind') & (verb_df["act/mid/p"] == 'act') & (verb_df["gender"] == " ") & (verb_df["case"] == " ") & (verb_df["person"] == "3rd") & (verb_df["number"] == analysis[6]) & (verb_df["dialects"] == " ")]

,lemma,form,tense,mode,act/mid/p,gender,case,person,number,dialects
300843,ἐνέπω,ἤνισπον,aor,ind,act,,,3rd,pl,


In [22]:
len(noun_dict["lemma"])

216252

In [26]:
len(verb_df)

859332